<a href="https://colab.research.google.com/github/GoogTech/langchain-tutorials/blob/master/LangGraph_Glossary/LangGraph_Glossary.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Refer to the langgraph tutorials of `v0.2.74` : https://langchain-ai.github.io/langgraph/concepts/low_level/

# Installation

In [ ]:
!pip install -U langgraph langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.3 MB/s eta 0:00:00


In [ ]:
!pip show langgraph langchain_openai

Name: langgraph
Version: 0.2.74
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-sdk
Required-by: 
---
Name: langchain-openai
Version: 0.3.6
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


# 1.Graphs

In short:
* nodes do the work.
* edges tell what to do next.
* state represents the memory that records what you did.

## 1.1.StateGraph

## 1.2.Compiling your graph

In [ ]:
graph = graph_builder.compile(...)

# 2.State

## 2.1.Schema

### 2.1.1.How to use Pydantic model as graph state

https://langchain-ai.github.io/langgraph/how-tos/state-model/

A `StateGraph` accepts a state_schema argument on initialization `that` specifies the "shape" of the state `that` the nodes in the graph can access and update.

**Input Validation：**

In [ ]:
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel

# The overall state of the graph(this is the public state shared across nodes)
class OverallState(BaseModel):
  a: str

def node(state: OverallState):
  return {"a": "goodbye"}

# Build the state graph:
builder = StateGraph(OverallState)
builder.add_node('node', node)
builder.add_edge(START, 'node')
builder.add_edge('node', END)
graph = builder.compile()

# Test the graph with a valid input
graph.invoke({"a": "hello"})

{'a': 'goodbye'}

Invoke the graph with an invalid input

In [ ]:
try:
  graph.invoke({"a": 123})
except Exception as e:
    print("An exception was raised because `a` is an integer rather than a string.\n")
    print(e)

An exception was raised because `a` is an integer rather than a string.

1 validation error for OverallState
a
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


**Multiple Nodes：**

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

from pydantic import BaseModel

# The overall state of the graph (this is the public state shared across nodes)
class OverallState(BaseModel):
    a: str

def bad_node(state: OverallState):
    return { # !!!returns an update to the state!!!
        "a": 123  # Invalid
    }

def ok_node(state: OverallState):
    return {"a": "goodbye"}

# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(bad_node)
builder.add_node(ok_node)
builder.add_edge(START, "bad_node")
builder.add_edge("bad_node", "ok_node")
builder.add_edge("ok_node", END)
graph = builder.compile()

# Test the graph with a valid input
try:
    graph.invoke({"a": "hello"}) # !!!the validation error will occur when ok_node is called!!!
except Exception as e:
    print("An exception was raised because bad_node sets `a` to an integer.")
    print(e)

An exception was raised because bad_node sets `a` to an integer.
1 validation error for OverallState
a
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


### 2.1.2.How to define input/output schema for your graph

https://langchain-ai.github.io/langgraph/how-tos/input_output_schema/

By default, StateGraph operates with a single schema, and all nodes are expected to communicate using that schema.

However, it's also possible to define distinct input and output schemas for a graph.

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# Define the schema for the input.
# The input schema ensures that the provided input
# matches the expected structure.
class InputState(TypedDict):
    question: str

# Define the schema for the output.
# The output schema filters the internal data to return
# only the relevant information according to the defined output schema.
class OutputState(TypedDict):
    answer: str

# Define the overall schema, combining both input and output
class OverallState(InputState, OutputState):
    pass

# Define the node that processes the input and generates an answer
def node(state: OverallState):
    return {"question": state["question"], "answer": "This is answer..."}

# Build the graph with input and output schemas specified
builder = StateGraph(OverallState, input=InputState, output=OutputState)
builder.add_node(node)
builder.add_edge(START, "node")
builder.add_edge("node", END)
graph = builder.compile()

# Invoke the graph with and input and print the result.
# Notice that the output of invoke only includes the output schema.
print(graph.invoke({"question": "What is the meaning of life?"}))

{'answer': 'This is answer...'}


### Multiple schemas

we define an "internal" schema that contains all keys relevant to graph operations.

But, we also define input and output schemas that are sub-sets of the "internal" schema to constrain the input and output of the graph.

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

class InputState(TypedDict):
  user_input: str # shared with the schema of `OverallState`

class OutputState(TypedDict):
  graph_output: str # shared with the schema of `OverallState`

class OverallState(InputState, OutputState):
  foo: str
  user_input: str
  graph_output: str

class PrivateState(TypedDict):
  bar: str # private variable, because any one of schema doesn't include it.

#################################################################################################
# Sate Process: User ————> InputState ——> OverallState ——> PrivateSate ——> OutputState ——> User #
#################################################################################################
def node_1(state: InputState) -> OverallState:
  # Read from InputState, write to OverallState
  return {"foo": state["user_input"] + " name"}

def node_2(state: OverallState) -> PrivateState:
  # Read from OverallState, write to PrivateState
  return {"bar": state["foo"] + " is"}

def node_3(state: PrivateState) -> OutputState:
  # Read from PrivateState, write to OutputState
  return {"graph_output": state["bar"] + " Lance"}

builder = StateGraph(OverallState, input=InputState, output=OutputState)
builder.add_node(node_1)
builder.add_node(node_2)
builder.add_node(node_3)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

graph = builder.compile()
graph.invoke({"user_input": "My"})

{'graph_output': 'My name is Lance'}

#### How to pass private state between nodes

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# The overall state of the graph(this is the public state shared across nodes)
class OverallState(TypedDict):
  a: str

# Output from node_1 contains private data that is not part of the overall state
class Node1Output(TypedDict):
  private_data: str

# Node 2 input onlu requests the private data available after node_1
class Node2Input(TypedDict):
  private_data: str

#################################################################################################################
# Sate Process: User ——> OverallState ——> Node1Output ~~~ Node2Input ——> OverallState ——> OverallState ——> User #
#################################################################################################################
# The private data is only shared between node_1 and node_2
def node_1(state: OverallState) -> Node1Output:
  output = {"private_data": "set by node_1"}
  print(f"Entered node `node_1`: \n\tInput: {state}.\n\tReturn:{output}")
  return output

def node_2(state: Node2Input) -> OverallState:
  output = {"a": "set by node_2"}
  print(f"Entered node `node_2`:\n\tInput: {state}.\n\tReturned: {output}")
  return output

# Node 3 only has access to the overall state (!!!no access to private data from node_1!!!)
def node_3(state: OverallState) -> OverallState:
  output = {"a": "set by node_3"}
  print(f"Entered node `node_3`:\n\tInput: {state}.\n\tReturned: {output}")
  return output

# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(node_1) # node_1 is the first node
builder.add_node(node_2) # node_2 is the second node and accepts private data from node_1
builder.add_node(node_3) # node_3 is the third node and does not see the private data
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

graph = builder.compile()

# Invoke the graph with the initial state
response = graph.invoke({"a": "hello"})
print(f"\nOutput of graph invocation: {response}")

Entered node `node_1`: 
	Input: {'a': 'hello'}.
	Return:{'private_data': 'set by node_1'}
Entered node `node_2`:
	Input: {'private_data': 'set by node_1'}.
	Returned: {'a': 'set by node_2'}
Entered node `node_3`:
	Input: {'a': 'set by node_2'}.
	Returned: {'a': 'set by node_3'}

Output of graph invocation: {'a': 'set by node_3'}


## 2.2.Reducers

Example A:

In [ ]:
from typing_extensions import TypedDict

class State(TypedDict):
    foo: int
    bar: list[str]

Example B:

In [ ]:
from typing import Annotated
from typing_extensions import  TypedDict
from operator import  add

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add]

## 2.3.Working with Messages in Graph State

### 2.3.1.Messages

LangChain provides a unified message format that can be used across all chat models, allowing users to work with different chat models without worrying about the specific details of the message format used by each model provider.

The five main message types are :

* SystemMessage: corresponds to `system` role
* HumanMessage: corresponds to `user` role
* AIMessage: corresponds to `assistant` role
* AIMessageChunk: corresponds to `assistant` role, used for `streaming` responses
* ToolMessage: corresponds to `tool` role

HumanMessage :

In [ ]:
from langchain_core.messages import HuamnMessage
model.invoke([HuamanMessage(content="hello, how are you?")])

# Or:
# When invoking a chat model with a string as input,
# LangChain will automatically convert the string into a HumanMessage object.
# This is mostly useful for quick testing.
model.invoke("hello, how are you?")

AIMessage :

In [ ]:
from langchain_core.messages import HumanMessage

ai_message = model.invoke([HumanMessage(content="Tell me a joke")])
ai_message # <-- AIMessage

AIMessageChunk :

In [ ]:
for chunk in model.stream([HuamnMessage("What color is the sky?")]):
  print(chunk)

OpenAI Format :
Chat models also accept OpenAI's format as inputs to chat models

In [ ]:
chat_model.invoke([
    {
      "role": "user",
      "content": "Hello, how are you?",
    },
    {
        "role": "assistant",
        "content": "I'm doing well, thank you for asking.",
    },
    {
        "role": "user",
        "content": "Can you tell me a joke?",
    }
])

### 2.3.2.Serialization

In [ ]:
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

class State(TypedDict):
  # the state updates are always deserialized into LangChain Messages
  # when using add_messages
  messages: Annotated[list[AnyMessage], add_messages]

### 2.3.3.MessagesState

In [ ]:
from langgraph.graph import MessagesState

# MessagesState is defined with a single messages key
# which is a list of AnyMessage objects and uses the add_messages reducer.
class State(MessagesState):
  # Typically, there is more state to track than just messages,
  # so we see people subclass this state and add more fields, like:
  documents: list[str]

# 3.Nodes

In [ ]:
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph

builder = StateGraph(dict)https://langchain-ai.github.io/langgraph/concepts/low_level/#start-node

# the first positional argument is the state,
# and (optionally), the second positional argument is a "config",
def my_node(state: dict, config: RunnableConfig):
  print("In node:", config["configurable"]["user_id"])
  return {"results": f"Hello, {state['input']}"}

# The second argument is optional
def my_other_node(state: dict):
  return state

builder.add_node('my_node', my_node)
# If you add a node to a graph without specifying a name,
# it will be given a default name equivalent to the function name.
builder.add_node(my_other_node)

### 3.1.START Node

In [ ]:
from langgraph.graph import START

#  The main purpose for referencing this node is
# to determine which nodes should be called first.
builder.add_edge(START, "my_node")

### 3.2.END Node

In [ ]:
from langgraph.graph import END

# This node is referenced when you want to denote
# which edges have no actions after they are done.
graph.add_edge("node_a", END)

# 4.Edges

### 4.1.Normal Edges


In [ ]:
graph.add_edge("node_a", "node_b")

### 4.2.Conditional Edges

In [ ]:
# By default, the return value routing_function is used as the name of the node (or list of nodes) to send the state to next.
# All those nodes will be run in parallel as a part of the next superstep.
graph.add_conditional_edges("node_a", routing_function)

# Or you can optionally provide a dictionary that maps the routing_function's output to the name of the next node.
graph.add_conditional_edges("node_a", routing_function, {True: "node_b", False: "node_a"})

# 5.Send

In [ ]:
def continue_to_jokes(state: OverallState):
  # Send takes two arguments:
  # first is the name of the node, and second is the state to pass to that node.
  return [Send("generate_joke", {"subject": s}) for s in state['subjects']]

graph.add_conditional_edges('node_a', continue_to_jokes)

# 6.Command

In [ ]:
#  you might want to BOTH perform state updates AND decide
# which node to go to next in the SAME node.
def my_node(state: State) -> Command[Literals["my_other_node"]]:
  return Command(
      # state udpate
      update = ("foo": "bar"),
      # control flow
      goto="my_other_node"
  )

# With `Command` you can also achieve dynamic control flow behavior (identical to conditional edges)
def my_node(state: State) -> Command[Literal["my_other_node"]]:
  if state["foo"] == "bar":
    return Command(
        update = {"foo": "baz"},
        goto = "my_other_node"
    )

## 6.1.When should I use Command instead of conditional edges?

Use Command when you need to both update the graph state and route to a different node.

For example, when implementing multi-agent handoffs where it's important to route to a different agent and pass some information to that agent.

## 6.2.Navigating to a node in a parent graph

In [ ]:
def my_node(state: State) -> Command[Literal['my_other_node']]:
  return Command(
      update = {"foo": "bar"},
      goto = "my_other_node", # # where `other_subgraph` is a node in the parent graph
      graph = Command.PARENT
  )

## 6.3.Using inside tools

In [ ]:
@tool
def lookup_user_info(tool_call_id: Annotated[str, InjectedToolCallId], config: RunnableConfig):
    """Use this to look up user information to better assist them with their questions."""
    user_info = get_user_info(config.get("configurable", {}).get("user_id"))
    return Command(
        update={
            # update the state keys
            "user_info": user_info,
            # update the message history
            "messages": [ToolMessage("Successfully looked up user information", tool_call_id=tool_call_id)]
        }
    )

# 7.Persistence

# 8.Threads

# 9.Storage

# 10.Graph Migrations

# 11.Configuration

# 12.interrupt

# 13.Breakpoints

# 14.Subgraphs

# 15.Visualization

# 16.Streaming